In [1]:
#Load required libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
#Loading and preparing databases

df_neutral = pd.read_csv('seac_neutral.csv')

df_neutral.drop('name',axis = 1,inplace = True)
df_neutral.drop('gender',axis = 1,inplace = True)
df_neutral.drop('age',axis = 1,inplace = True)
n_speakers_neutral = len(df_neutral['class'].unique())

df_anger = pd.read_csv('seac_anger.csv')

df_anger.drop('name',axis = 1,inplace = True)
df_anger.drop('gender',axis = 1,inplace = True)
df_anger.drop('age',axis = 1,inplace = True)
n_speakers_anger = len(df_anger['class'].unique())

df_fear = pd.read_csv('seac_fear.csv')

df_fear.drop('name',axis = 1,inplace = True)
df_fear.drop('gender',axis = 1,inplace = True)
df_fear.drop('age',axis = 1,inplace = True)
n_speakers_fear = len(df_fear['class'].unique())

df_sadness = pd.read_csv('seac_sadness.csv')

df_sadness.drop('name',axis = 1,inplace = True)
df_sadness.drop('gender',axis = 1,inplace = True)
df_sadness.drop('age',axis = 1,inplace = True)
n_speakers_sadness = len(df_sadness['class'].unique())

df_joy = pd.read_csv('seac_joy.csv')

df_joy.drop('name',axis = 1,inplace = True)
df_joy.drop('gender',axis = 1,inplace = True)
df_joy.drop('age',axis = 1,inplace = True)
n_speakers_joy = len(df_joy['class'].unique())

n_features = df_neutral.shape[1]

print("The number of speakers in an emotionally neutral state is:",n_speakers_neutral)
print("The number of speakers in a state of anger is:",n_speakers_anger)
print("The number of speakers is in a state of fear:",n_speakers_fear)
print("The number of speakers in a state of sadness is:",n_speakers_sadness)
print("The number of speakers in a state of joy is:",n_speakers_joy)

The number of speakers in an emotionally neutral state is: 55
The number of speakers in a state of anger is: 32
The number of speakers is in a state of fear: 31
The number of speakers in a state of sadness is: 30
The number of speakers in a state of joy is: 47


In [3]:
#Division of bases into training and test sets (training sets only exist for neutral base)

y_neutral = df_neutral['class']
x_neutral = df_neutral.drop(['class'], axis = 1)
mfcc_cols_neutral = [col for col in df_neutral.columns if 'mfcc' in col]
mfcc_x_neutral = x_neutral.loc[:,mfcc_cols_neutral]
x_neutral_train, x_neutral_test, y_neutral_train, y_neutral_test = train_test_split(mfcc_x_neutral, y_neutral, test_size=0.1, 
                                                                                    random_state=10, stratify=y_neutral)

y_anger = df_anger['class']
x_anger = df_anger.drop(['class'], axis = 1)
# mfcc_cols_anger = [col for col in df_anger.columns if 'mfcc' in col]
x_anger = x_anger.loc[:,mfcc_cols_neutral]

y_fear = df_fear['class']
x_fear = df_fear.drop(['class'], axis = 1)
# mfcc_cols_fear = [col for col in df_fear.columns if 'mfcc' in col]
x_fear = x_fear.loc[:,mfcc_cols_neutral]

y_sadness = df_sadness['class']
x_sadness = df_sadness.drop(['class'], axis = 1)
# mfcc_cols_tuga = [col for col in df_tuga.columns if 'mfcc' in col]
x_sadness = x_sadness.loc[:,mfcc_cols_neutral]

y_joy = df_joy['class']
x_joy = df_joy.drop(['class'], axis = 1)
# mfcc_cols_joy = [col for col in df_joy.columns if 'mfcc' in col]
x_joy = x_joy.loc[:,mfcc_cols_neutral]

In [4]:
#Standardization of values

s_neutral = StandardScaler()
s_neutral.fit(x_neutral_train)
x_neutral_train = s_neutral.transform(x_neutral_train)
x_neutral_test = s_neutral.transform(x_neutral_test)

x_anger = s_neutral.transform(x_anger)
x_fear = s_neutral.transform(x_fear)
x_sadness = s_neutral.transform(x_sadness)
x_joy = s_neutral.transform(x_joy)

In [5]:
#Dimensionality reduction by LDA method

lda_neutral = LinearDiscriminantAnalysis()
x_neutral_train_lda = lda_neutral.fit_transform(x_neutral_train, y_neutral_train)
x_neutral_test_lda = lda_neutral.transform(x_neutral_test)

x_anger_lda = lda_neutral.transform(x_anger)
x_fear_lda = lda_neutral.transform(x_fear)
x_sadness_lda = lda_neutral.transform(x_sadness)
x_joy_lda = lda_neutral.transform(x_joy)

lda_dimensions = x_neutral_train_lda.shape[1]

In [6]:
#Memory of results

results = pd.DataFrame(columns = ['classifier',
                                    'reduction of dimensionality', 
                                    'accuracy [%] - neutral',
                                    'accuracy [%] - anger',
                                    'accuracy [%] - fear',
                                    'accuracy [%] - sadness',
                                    'accuracy [%] - joy',
                                    'recall [%] - neutral',
                                    'recall [%] - anger',
                                    'recall [%] - fear',
                                    'recall [%] - sadness',
                                    'recall [%] - joy',
                                    'precision [%] - neutral',
                                    'precision [%] - anger',
                                    'precision [%] - fear',
                                    'precision [%] - sadness',
                                    'precision [%] - joy',
                                    'f score [%] - neutral',
                                    'f score [%] - anger',
                                    'f score [%] - fear',
                                    'f score [%] - sadness',
                                    'f score [%] - joy',
                                    'number of dimensions'])

In [7]:
#Klasifikacija

for i in range(3):
    
    if i == 0:
        classifier = SVC(C = 1.0, decision_function_shape = 'ovo', degree = 3, gamma = 'scale', kernel = 'rbf')
        classifier_type = 'SVM'
        reduction_of_dimensionality = 'LDA'
        n_dimensions = lda_dimensions
    elif i == 1:
        classifier = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
        classifier_type = 'kNN'
        reduction_of_dimensionality = 'LDA'
        n_dimensions = lda_dimensions
    elif i == 2:
        classifier = MLPClassifier(activation= 'relu', learning_rate_init= 0.01, max_iter= 200, n_iter_no_change= 5, solver='adam')
        classifier_type = 'NN'
        reduction_of_dimensionality = 'LDA'
        n_dimensions = lda_dimensions
        
    classifier.fit(x_neutral_train_lda, y_neutral_train)

    y_neutral_pred = classifier.predict(x_neutral_test_lda)
    y_anger_pred = classifier.predict(x_anger_lda)
    y_fear_pred = classifier.predict(x_fear_lda)
    y_sadness_pred = classifier.predict(x_sadness_lda)
    y_joy_pred = classifier.predict(x_joy_lda)

    accuracy_neutral = round(accuracy_score(y_neutral_test, y_neutral_pred)*100,3)
    recall_neutral = round(recall_score(y_neutral_test, y_neutral_pred, average='macro')*100,3)
    precision_neutral = round(precision_score(y_neutral_test, y_neutral_pred, average='macro')*100,3)
    f_score_neutral = round(f1_score(y_neutral_test, y_neutral_pred, average='macro')*100,3)

    accuracy_anger = round(accuracy_score(y_anger, y_anger_pred)*100,3)
    recall_anger = round(recall_score(y_anger, y_anger_pred, average='macro')*100,3)
    precision_anger = round(precision_score(y_anger, y_anger_pred, average='macro')*100,3)
    f_score_anger = round(f1_score(y_anger, y_anger_pred, average='macro')*100,3)

    accuracy_fear = round(accuracy_score(y_fear, y_fear_pred)*100,3)
    recall_fear = round(recall_score(y_fear, y_fear_pred, average='macro')*100,3)
    precision_fear = round(precision_score(y_fear, y_fear_pred, average='macro')*100,3)
    f_score_fear = round(f1_score(y_fear, y_fear_pred, average='macro')*100,3)

    accuracy_sadness = round(accuracy_score(y_sadness, y_sadness_pred)*100,3)
    recall_sadness = round(recall_score(y_sadness, y_sadness_pred, average='macro')*100,3)
    precision_sadness = round(precision_score(y_sadness, y_sadness_pred, average='macro')*100,3)
    f_score_sadness = round(f1_score(y_sadness, y_sadness_pred, average='macro')*100,3)

    accuracy_joy = round(accuracy_score(y_joy, y_joy_pred)*100,3)
    recall_joy = round(recall_score(y_joy, y_joy_pred, average='macro')*100,3)
    precision_joy = round(precision_score(y_joy, y_joy_pred, average='macro')*100,3)
    f_score_joy = round(f1_score(y_joy, y_joy_pred, average='macro')*100,3)
    
    data = {'classifier': classifier_type,
            'reduction of dimensionality': reduction_of_dimensionality,
            'accuracy [%] - neutral': accuracy_neutral,
            'accuracy [%] - anger': accuracy_anger,
            'accuracy [%] - fear': accuracy_fear,
            'accuracy [%] - sadness': accuracy_sadness,
            'accuracy [%] - joy': accuracy_joy,
            'recall [%] - neutral': recall_neutral,
            'recall [%] - anger': recall_anger,
            'recall [%] - fear': recall_fear,
            'recall [%] - sadness': recall_sadness,
            'recall [%] - joy': recall_joy,
            'precision [%] - neutral': precision_neutral,
            'precision [%] - anger': precision_anger,
            'precision [%] - fear': precision_fear,
            'precision [%] - sadness': precision_sadness,
            'precision [%] - joy': precision_joy,
            'f score [%] - neutral': f_score_neutral,
            'f score [%] - anger': f_score_anger,
            'f score [%] - fear': f_score_fear,
            'f score [%] - sadness': f_score_sadness,
            'f score [%] - joy': f_score_joy,
            'number of dimensions':n_dimensions}

    results = results.append(data, ignore_index=True)

C:\Users\igorm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\igorm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\igorm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\igorm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision 

In [8]:
#Display results by accuracy

results_accuracy = results[['classifier','reduction of dimensionality','accuracy [%] - neutral','accuracy [%] - anger',
                            'accuracy [%] - fear','accuracy [%] - sadness','accuracy [%] - joy','number of dimensions']]

results_accuracy.head()

,classifier,reduction of dimensionality,accuracy [%] - neutral,accuracy [%] - anger,accuracy [%] - fear,accuracy [%] - sadness,accuracy [%] - joy,number of dimensions
0,SVM,LDA,100.0,56.73,63.555,59.051,59.772,54
1,kNN,LDA,99.704,57.003,59.586,54.617,59.241,54
2,NN,LDA,99.704,54.605,57.965,56.182,58.14,54


In [9]:
#Display results by recall

results_recall = results[['classifier','reduction of dimensionality','recall [%] - neutral','recall [%] - anger',
                           'recall [%] - fear','recall [%] - sadness','recall [%] - joy','number of dimensions']]

results_recall.head()

,classifier,reduction of dimensionality,recall [%] - neutral,recall [%] - anger,recall [%] - fear,recall [%] - sadness,recall [%] - joy,number of dimensions
0,SVM,LDA,100.0,35.865,39.168,35.281,54.307,54
1,kNN,LDA,99.697,35.272,37.638,31.984,50.72,54
2,NN,LDA,99.697,31.808,33.389,32.38,51.408,54


In [10]:
#Display results by precision

results_precision = results[['classifier','reduction of dimensionality','precision [%] - neutral','precision [%] - anger',
                             'precision [%] - fear','precision [%] - sadness','precision [%] - joy','number of dimensions']]

results_precision.head()

,classifier,reduction of dimensionality,precision [%] - neutral,precision [%] - anger,precision [%] - fear,precision [%] - sadness,precision [%] - joy,number of dimensions
0,SVM,LDA,100.0,45.214,46.605,45.515,62.066,54
1,kNN,LDA,99.74,43.29,45.934,40.664,55.571,54
2,NN,LDA,99.773,39.182,41.107,41.407,56.476,54


In [11]:
#Display results by f score

results_f_score = results[['classifier','reduction of dimensionality','f score [%] - neutral','f score [%] - anger',
                           'f score [%] - fear','f score [%] - sadness','f score [%] - joy','number of dimensions']]

results_f_score.head()

,classifier,reduction of dimensionality,f score [%] - neutral,f score [%] - anger,f score [%] - fear,f score [%] - sadness,f score [%] - joy,number of dimensions
0,SVM,LDA,100.0,35.979,39.657,35.999,51.434,54
1,kNN,LDA,99.695,35.573,38.496,32.658,47.542,54
2,NN,LDA,99.713,32.53,34.302,33.283,48.398,54
